In [1]:
import runpy

In [2]:
file_globals = runpy.run_path("../run_generation.py")
sample_sequence = file_globals['sample_sequence']

In [3]:
yt_encoder = runpy.run_path("../yt_encoder.py")
YTEncoder = yt_encoder['YTEncoder']

In [4]:
from transformers import GPT2LMHeadModel
import threading
import regex as re

import logging

logging.basicConfig(filename="rest.log", level=logging.INFO)
logger = logging.getLogger(__name__)

from os import environ
device = environ.get('DEVICE', 'cuda:0')

model_path = '../gpt2/medium'

tokenizer = YTEncoder.from_pretrained(model_path)

poetry_model = GPT2LMHeadModel.from_pretrained(model_path + '/poetry')
poetry_model.to(device)
poetry_model.eval()

def get_sample(model, prompt, length:int, num_samples:int, allow_linebreak:bool):
    logger.info("*" * 200)
    logger.info(prompt)
   
    filter_n = tokenizer.encode('\n')[-1:]
    filter_single = [1] 
    filter_single += [] if allow_linebreak else filter_n

    context_tokens = tokenizer.encode(prompt)
    out = sample_sequence(
        model=model,
        context=context_tokens,
        length=length,
        temperature=1,
        top_k=0,
        top_p=0.9,
        device=device,
        filter_single=filter_single,
        filter_double=filter_n,
        num_samples=num_samples,
    ).to('cpu')

    prompt = tokenizer.decode(context_tokens)
    len_prompt = len(prompt)
   
    replies = [out[item, :].tolist() for item in range(len(out))]
    text = [tokenizer.decode(item)[len_prompt:] for item in replies]
    reg_text = [re.match(r'[\w\W]*[\.!?]\n', item) for item in text]
    result = [reg_item[0] if reg_item else item  for reg_item, item in zip(reg_text,text)]
    logger.info("=" * 200)
    logger.info(result)
    return result

12/22/2019 09:48:13 - INFO - transformers.configuration_utils -   loading configuration file ../gpt2/medium/poetry/config.json
12/22/2019 09:48:13 - INFO - transformers.configuration_utils -   Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "is_decoder": false,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "predict_special_tokens": true,
  "pruned_heads": {},
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 50257
}

12/22/2019 09:48:13 - INFO - transformers.modeling_utils -   loading weights file ../gpt2/medium/poetry/pytorch_model.b

In [42]:
from rupo.api import Engine
from rupo.rhymes.rhymes import Rhymes

In [6]:
engine = Engine(language="ru")

In [7]:
engine.is_rhyme("корова", "здорова")

12/22/2019 09:48:27 - INFO - allennlp.common.params -   vocab = None
12/22/2019 09:48:27 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /home/u/.anaconda/envs/gpt/lib/python3.7/site-packages/russ/models/ru-main/vocabulary.
12/22/2019 09:48:27 - INFO - allennlp.common.registrable -   instantiating registered subclass base of <class 'allennlp.models.model.Model'>
12/22/2019 09:48:27 - INFO - allennlp.common.params -   type = default
12/22/2019 09:48:27 - INFO - allennlp.common.registrable -   instantiating registered subclass default of <class 'allennlp.data.vocabulary.Vocabulary'>
12/22/2019 09:48:27 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /home/u/.anaconda/envs/gpt/lib/python3.7/site-packages/russ/models/ru-main/vocabulary.
12/22/2019 09:48:27 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {'dense_dim': 128, 'embedder': {'token_embedders': {'tokens': {'embedding_dim': 32, 't

True

In [8]:
engine.is_rhyme('привет', 'ответ')

True

In [9]:
engine.is_rhyme('пизда', 'джигурда')

False

In [10]:
from transitions import Machine

In [11]:
def get_sample(model, prompt, length:int, num_samples:int, allow_linebreak:bool):
    logger.info("*" * 200)
    logger.info(prompt)
   
    filter_n = tokenizer.encode('\n')[-1:]
    filter_single = [1] 
    filter_single += [] if allow_linebreak else filter_n

    context_tokens = tokenizer.encode(prompt)
    out = sample_sequence(
        model=model,
        context=context_tokens,
        length=length,
        temperature=1,
        top_k=0,
        top_p=0.9,
        device=device,
        filter_single=filter_single,
        filter_double=filter_n,
        num_samples=num_samples,
    ).to('cpu')

    prompt = tokenizer.decode(context_tokens)
    len_prompt = len(prompt)
   
    replies = [out[item, :].tolist() for item in range(len(out))]
    text = [tokenizer.decode(item)[len_prompt:] for item in replies]
    reg_text = [re.match(r'[\w\W]*[\.!?]\n', item) for item in text]
    result = [reg_item[0] if reg_item else item  for reg_item, item in zip(reg_text,text)]
    logger.info("=" * 200)
    logger.info(result)
    return result


In [12]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import trange

from transformers import (GPT2Config, OpenAIGPTConfig, XLNetConfig, TransfoXLConfig, 
                                    GPT2LMHeadModel, GPT2Tokenizer, 
                                    OpenAIGPTLMHeadModel, OpenAIGPTTokenizer, 
                                    XLNetLMHeadModel, XLNetTokenizer, 
                                    TransfoXLLMHeadModel, TransfoXLTokenizer, )


logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

ALL_MODELS = sum((tuple(conf.pretrained_config_archive_map.keys()) for conf in (GPT2Config, OpenAIGPTConfig, XLNetConfig, TransfoXLConfig)), ())

MODEL_CLASSES = {
    'gpt2': (GPT2LMHeadModel, GPT2Tokenizer),
    'openai-gpt': (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    'xlnet': (XLNetLMHeadModel, XLNetTokenizer),
    'transfo-xl': (TransfoXLLMHeadModel, TransfoXLTokenizer),
}

FILTER_VALUE=-float('Inf')


In [13]:
model = poetry_model
length = 150
prompt = 'мы с иваном ильичем ехали на дизеле\nон мудак и я мудак\n'
context = tokenizer.encode(prompt)
num_samples = 1
temperature=1
top_k=0
top_p=0.0
is_xlnet=False
device='cuda'
max_input=1023
filter_single=[]
filter_double=[]

In [14]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


In [15]:
if '\n' not in prompt:
    initial = 'firstline'
else:
    initial = 'midline'

In [16]:
states = ['firstline', 'midline', 'rhymed', 'del']

transitions = [
    {'trigger': 'newline', 'source': 'firstline', 'dest': 'midline'},
    {'trigger': 'newline', 'source': 'midline',   'dest': 'del'},
    {'trigger': 'newline', 'source': 'rhymed',    'dest': 'midline'},
    {'trigger': 'rhyme',   'source': 'firstline', 'dest': 'del'},
    {'trigger': 'rhyme',   'source': 'midline',   'dest': 'rhymed'},
    {'trigger': 'rhyme',   'source': 'rhymed',    'dest': 'del'},
    {'trigger': 'word',    'source': 'firstline', 'dest': 'firstline'},
    {'trigger': 'word',    'source': 'rhymed',    'dest': 'del'},
    {'trigger': 'word',    'source': 'midline',   'dest': 'midline'},
    
    {'trigger': 'newline', 'source': 'del', 'dest': 'del'},
    {'trigger': 'rhyme',   'source': 'del', 'dest': 'del'},
    {'trigger': 'word',    'source': 'del', 'dest': 'del'},
]

class Stih(object):
    def __init__(self, text=''):
        self.tokens = tokenizer.encode(text)
        self.text = text
        self.p = 1
        
        if '\n' not in text:
            initial = 'firstline'
        else:
            initial = 'midline'
        self.machine = Machine(model=self, states=states, transitions=transitions, initial=initial)
    
    def add(self, token, p):
        token = int(token)
        self.tokens += [token]
        self.last = tokenizer.decode([token])
        self.text = tokenizer.decode(self.tokens)
        self.p *= p
        
        if '\n' in self.last:
            self.newline()
            
        if re.search(r'(\w)', self.last):
            self.word()
        return self.last
    
    def __repr__(self):
        return self.text

In [17]:
stih = Stih(prompt)

In [18]:
import copy

In [19]:
def filter_rhyme(stih, filtered_logits):
    def check_rhyme(token):
        last = tokenizer.decode([int(token)])
        if not re.search(r'([а-яА-Я])', last):
            return False

        total_stih = tokenizer.decode(stih.tokens + [token])
        last_word = re.findall(r'\b\w+\b', total_stih)[-1] 
        lines = total_stih.splitlines()
        rhyme_line = lines[max(-3, -len(lines))]
        rhyme_word = re.findall(r'\b\w+\b', rhyme_line)[-1]
        #print(last_word, rhyme_word)
        return engine.is_rhyme(last_word, rhyme_word)  
    
    arr = np.array(np.argsort(filtered_logits.cpu()))[::-1][:6400]
    rhymes = Pool(64).map(check_rhyme, arr)
    
    for itoken in arr:
        if filtered_logits[itoken] > FILTER_VALUE:
            if rhymes[itoken]:
                return itoken

In [60]:
from mezmorize import Cache

cache = Cache(CACHE_TYPE='filesystem', CACHE_DIR='cache')

@cache.memoize()
def is_rhyme(word1, word2):
    #return engine.is_rhyme(word1, word2) 
    markup_word1 = engine.get_markup(word1).lines[0].words[0]
    markup_word1.set_stresses(engine.get_stresses(word1))
    markup_word2 = engine.get_markup(word2).lines[0].words[0]
    markup_word2.set_stresses(engine.get_stresses(word2))
    return Rhymes.is_rhyme(markup_word1, markup_word2, 5)

In [61]:
def check_rhyme(stih, token):
    last = tokenizer.decode([int(token)])
    if not re.search(r'([а-яА-Я])', last):
        return False

    total_stih = tokenizer.decode(stih.tokens + [token])
    last_word = re.findall(r'\b\w+\b', total_stih)[-1] 
    lines = total_stih.splitlines()
    rhyme_line = lines[max(-3, -len(lines))]
    rhyme_word = re.findall(r'\b\w+\b', rhyme_line)[-1]
    if rhyme_word == last_word: return False
    
    is_rhymed = is_rhyme(last_word, rhyme_word) 
    if is_rhymed: print(last_word, rhyme_word)
    return is_rhymed


In [62]:
from profilehooks import profile

In [63]:
#@profile(immediate=True)
def filter_rhyme(stih, filtered_logits):
    for itoken in np.array(np.argsort(filtered_logits.cpu()))[::-1][:64000]:
        if filtered_logits[itoken] > 0:
            if check_rhyme(stih, itoken):
                return itoken

In [64]:
def filter_after_rhyme(stih, filtered_logits):
    for itoken in np.array(np.argsort(filtered_logits.cpu()))[::-1][:64000]:
        if filtered_logits[itoken] > 0:
            if not re.search(r'(\w)', tokenizer.decode([int(itoken)])):
                return itoken

In [65]:
stih.text

'мы с иваном ильичем ехали на дизеле\nон мудак и я мудак\n'

In [66]:
    stihi = [stih]
    with torch.no_grad():
        for _ in trange(length):
            context = torch.tensor([stih.tokens for stih in stihi], dtype=torch.long, device=device)
            inputs = {'input_ids': context[:,-max_input:]}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            next_tokens = torch.zeros(num_samples, dtype=torch.long).to(device)
            n_stihi = []
            for isample in range(len(stihi)):
                next_token_logits = outputs[0][isample, -1, :] / temperature

                next_token_logits[filter_single] = FILTER_VALUE
                # filter blank line = double \n
                if context[isample, -1] in filter_double:
                    next_token_logits[generated[isample, -1]] = FILTER_VALUE

                filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
                stih1 = copy.deepcopy(stihi[isample])
                
                if stih1.state != 'rhymed':
                    i_logit = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
                else:
                    i_logit = filter_after_rhyme(stih, filtered_logits)
                i_logit = int(i_logit)
                stih1.add(i_logit, filtered_logits[i_logit])
                n_stihi += [stih1]
                
                stih2 = copy.deepcopy(stihi[isample])
                stih2.rhyme()
                if stih2.state != 'del':
                    rhymed_logit = filter_rhyme(stih2, filtered_logits)
                    print(rhymed_logit)
                    if rhymed_logit:
                        stih2.add(rhymed_logit, filtered_logits[rhymed_logit])
                        stih2.machine.set_state('rhymed')
                        n_stihi += [stih2]
            stihi = [stih for stih in n_stihi if stih.state != 'del']
            stihi = sorted(stihi, key=lambda stih: stih.p, reverse=True)[:64]
            print(stihi)

  0%|          | 0/150 [00:00<?, ?it/s]12/22/2019 16:55:08 - INFO - transitions.core -   Exited state midline
12/22/2019 16:55:08 - INFO - transitions.core -   Entered state midline
12/22/2019 16:55:08 - INFO - transitions.core -   Exited state midline
12/22/2019 16:55:08 - INFO - transitions.core -   Entered state rhymed
  1%|          | 1/150 [03:28<8:37:37, 208.44s/it]12/22/2019 16:58:37 - INFO - transitions.core -   Exited state midline
12/22/2019 16:58:37 - INFO - transitions.core -   Entered state midline
12/22/2019 16:58:37 - INFO - transitions.core -   Exited state midline
12/22/2019 16:58:37 - INFO - transitions.core -   Entered state rhymed


None
[мы с иваном ильичем ехали на дизеле
он мудак и я мудак
Идет]


  1%|▏         | 2/150 [06:18<8:05:59, 197.02s/it]12/22/2019 17:01:27 - INFO - transitions.core -   Exited state midline
12/22/2019 17:01:27 - INFO - transitions.core -   Entered state midline
12/22/2019 17:01:27 - INFO - transitions.core -   Exited state midline
12/22/2019 17:01:27 - INFO - transitions.core -   Entered state rhymed


None
[мы с иваном ильичем ехали на дизеле
он мудак и я мудак
Идет полдень]


  1%|▏         | 2/150 [08:22<10:19:35, 251.18s/it]


KeyboardInterrupt: 

In [389]:
n_stihi[0].text

'мы с иваном ильичем ехали на дизеле\nКак я тебя любил!\n'

In [189]:
print(stih.text)

мы с иваном ильичем ехали на дизеле
Горе, гнев, тревога, гарь,
Пока из прорехи распахнутой конторы не выбежал,
Средь дыма, пыли, грохота, вдох,
И, бодрый не меньшеприбыльный, человек
В кузове грузовика им. Жуковского одного.

Молчаливый, упрямый,
Он любил с детства беседы с природой.
Сколько тонительных напитков,
Сколько красного вина,
Сколько винных капелек оказалось -
Создателем не предназначено.
Сколько буйных чувств! Сколько соков.
Сколько смеха и слез,
Сколько сладкого щебета!
Какие красивые краски...
Какая прелесть
Розовый, фиолетовый, голубой!
И мягкая пружина,
Ободряющий, щекочущий.
Но радость, отрава эта,
Как, наверное, горька,
Возникла в просто


In [144]:
print(tokenizer.decode(generated.to('cpu')))

мы с иваном ильичем ехали на дизеле с командующим на полуторке.
И пригорок и кустик
Осенней упорною петропламенкой
облиты белизною, тяжелою.
Но зато горы та ж мудрель,
и Ташкент с их снежными улыбками
не забудет педелю беготни
солдатиков-сыновей.
Сейчас министры ушли,
улетели, замаливая
Родина при людях и при детях
бедами суровыми.
Не слышит пыль, им принадлежащую,
солдатский разговор солдатский,
что в небе много звезд надоевшей.
Поняв, что счастье его
тоже детское.
За такие торжественные штуки
смотрят на него в штыки.
Ну а молчание за ними
так прочно, что, может быть,
даже есть какое-над
60
? съ
и…! –!;
- –? Иль!ке,ке.
...!; в;
назад, с в
своем
;
?!
в Де и варен
во в
в и —!й, с
и
!
на!
по

:
;



шке, раке
Мо

;

сверка

к

в


... –; «по
Ю пло; по!"?
том
по:

— досте,...!; зеленом "

в
в в! оригин с
вместе
в

на




всю
своем.: —
на


чке «рез


In [7]:
get_sample(poetry_model, 'тест', 50, 1, True)

12/21/2019 11:02:19 - INFO - __main__ -   ********************************************************************************************************************************************************************************************************
12/21/2019 11:02:19 - INFO - __main__ -   тест
100%|██████████| 50/50 [00:01<00:00, 37.81it/s]
12/21/2019 11:02:20 - INFO - __main__ -   ========================================================================================================================================================================================================
12/21/2019 11:02:20 - INFO - __main__ -   ['ю,\nПодымать от зерна..."\nИ день как в пламя врезался...\nИстомившись, ко сну...\n']


['ю,\nПодымать от зерна..."\nИ день как в пламя врезался...\nИстомившись, ко сну...\n']